In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Credit_Card_Fault_Prediction'

In [12]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: List[str]
    x_test_data_path: List[str]
    y_test_data_path: List[str]
    evaluation_report: List[str]

In [8]:
from CreditCardFraudDetection.constants import (
    PARAMS_YAML_FILE_PATH, CONFIG_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
)
from CreditCardFraudDetection.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(self,
                 params_yaml_file_path = PARAMS_YAML_FILE_PATH,
                 config_yaml_file_path = CONFIG_YAML_FILE_PATH,
                 schema_yaml_file_path = SCHEMA_YAML_FILE_PATH):
        
        self.params = read_yaml(params_yaml_file_path)
        self.config = read_yaml(config_yaml_file_path)
        self.schema = read_yaml(schema_yaml_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            model_path = config.model_path,
            x_test_data_path = config.x_test_data_path,
            y_test_data_path = config.y_test_data_path,
            evaluation_report = config.evaluation_report
        )

In [14]:
import pandas as pd 
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import os
from CreditCardFraudDetection import logger

In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def calculate_metrics(self, y_true, y_pred, y_prob):
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        roc_auc = roc_auc_score(y_true, y_prob)

        return accuracy, precision, recall, f1, roc_auc
    
    def evaluate_model(self, model_number: int):
        assert model_number in [1, 2], "Invalid model number, It should be 1 or 2"

        X_test = np.load(self.config.x_test_data_path[model_number - 1])
        y_test = np.load(self.config.y_test_data_path[model_number - 1])
        logger.info(f"Loaded test data for model {model_number}")

        model = joblib.load(self.config.model_path[model_number - 1])
        logger.info(f"Loaded model {model_number}")

        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]
        logger.info(f"Predictions for model {model_number} done")

        accuracy, precision, recall, f1, roc_auc = self.calculate_metrics(y_test, y_pred, y_prob)
        logger.info(f"Metrics calculated for model {model_number}")

        scores = {
            "accuracy_score": accuracy,
            "precision_score": precision,
            "recall_score": recall,
            "f1_score": f1,
            "roc_auc_score": roc_auc
        }

        if not os.path.exists(self.config.evaluation_report[model_number - 1]):
            os.makedirs(self.config.evaluation_report[model_number - 1])
            
        save_json(path = Path(os.path.join(self.config.evaluation_report[model_number - 1], "metrics.json")), data = scores)
        np.save(os.path.join(self.config.evaluation_report[model_number - 1], "y_pred.npy"), y_pred)
        np.save(os.path.join(self.config.evaluation_report[model_number - 1], "y_prob.npy"), y_prob)
        logger.info(f"Metrics saved for model {model_number}")

        

In [20]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)

    model_evaluation.evaluate_model(model_number = 1)
    model_evaluation.evaluate_model(model_number = 2)

except Exception as e:
    logger.error(e)
    raise e

[2024-07-05 21:05:14,757: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 21:05:14,762: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-05 21:05:14,766: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-05 21:05:14,766: INFO: common: created directory at: artifacts]
[2024-07-05 21:05:14,767: INFO: common: created directory at: artifacts/model_evaluation]
[2024-07-05 21:05:14,776: INFO: 3089079914: Loaded test data for model 1]
[2024-07-05 21:05:14,791: INFO: 3089079914: Loaded model 1]
[2024-07-05 21:06:15,984: INFO: 3089079914: Predictions for model 1 done]
[2024-07-05 21:06:16,146: INFO: 3089079914: Metrics calculated for model 1]
[2024-07-05 21:06:16,147: INFO: common: json file saved at: artifacts/model_evaluation/fraudulent_transection/metrics.json]
[2024-07-05 21:06:16,155: INFO: 3089079914: Metrics saved for model 1]
[2024-07-05 21:06:16,159: INFO: 3089079914: Loaded test data for model 2]
[2024-07-05 21:06:16,174: